In [1]:
pip install yfinance numpy pandas matplotlib scikit-learn tensorflow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# model.py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
import os

In [4]:
# Set your desired save directory
base_dir = r"C:\Users\lenovo\OneDrive\Desktop\intership project\Stock Price Prediction (LSTM)"
model_path = os.path.join(base_dir, "stock_lstm_model.h5")
plot_path = os.path.join(base_dir, "static", "plot.png")
os.makedirs(os.path.dirname(plot_path), exist_ok=True)

In [5]:

def predict_stock(ticker='AAPL'):
    # Download historical stock data
    df = yf.download(ticker, start='2015-01-01', end='2024-12-31')
    data = df[['Close']]
    
    # Scale data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)

    # Create sequences
    X, y = [], []
    for i in range(60, len(scaled_data)):
        X.append(scaled_data[i-60:i, 0])
        y.append(scaled_data[i, 0])
    X, y = np.array(X), np.array(y)
    X = X.reshape((X.shape[0], X.shape[1], 1))

    # Train/Test Split
    split = int(len(X) * 0.8)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # Load existing model or train new
    if os.path.exists(model_path):
        model = load_model(model_path)
    else:
        model = Sequential()
        model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
        model.add(LSTM(50))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)
        model.save(model_path)  # Save model to specified path

    # Predict
    predicted_prices = model.predict(X_test)
    predicted_prices = scaler.inverse_transform(predicted_prices.reshape(-1, 1))
    real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Plot
    plt.figure(figsize=(10, 5))
    plt.plot(real_prices, label='Actual')
    plt.plot(predicted_prices, label='Predicted')
    plt.title(f'{ticker} Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend()
    plt.savefig(plot_path)
    plt.close()

    # Predict next day
    last_60_days = scaled_data[-60:].reshape(1, 60, 1)
    next_day = model.predict(last_60_days)
    next_day = scaler.inverse_transform(next_day)

    return round(next_day[0][0], 2), plot_path


In [6]:
 predict_stock()

C:\Users\lenovo\AppData\Local\Temp\ipykernel_15948\980684524.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start='2015-01-01', end='2024-12-31')
[*********************100%***********************]  1 of 1 completed
C:\Users\lenovo\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


(np.float32(238.86),
 'C:\\Users\\lenovo\\OneDrive\\Desktop\\intership project\\Stock Price Prediction (LSTM)\\static\\plot.png')